In [5]:
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
import sys

def euclidean_norm(vec):
    res = 0
    for el in vec:
        res += el**2
    return np.sqrt(res)

# сравнение по евклидовой норме
def get_diff(x1, x2):
    x = []
    for i in range(0, len(x1)):
        x.append(abs(x1[i] - x2[i]))
    return euclidean_norm(x)

# согласованная матричная норма
def matrix_norm(matrix):
  sum = 0
  for i in range(len(matrix)):
    sum += abs(matrix[i])
  return max(sum)

def generate_symmetrical_matrix(l, r, n):
    a = np.random.uniform(l, r, (n, n))
    a = np.tril(a) + np.tril(a, -1).T
    return a

def generate_vec(l, r, n):
  vec = np.random.uniform(l, r, n)
  return vec

def increase_diag_elems(a, diag):
    n = len(a)
    for i in range(0, len(a)):
        a[i][i] = diag * sum(abs(a[i][j]) if j != i else 0 for j in range(n))
    return a

# Вычисляем диагональное преобладание
# Проверить выполнение условия диагонального преобладания
def calc_diagonal_dominance(a):
  degree = max(abs(a[i][i]) - sum(abs(a[i][j]) if j != i else 0 for j in range(len(a))) for i in range(len(a)))
  return degree > 0

def LU(A):
    # A = L * U
    n = len(A)
    L, U = np.zeros_like(A), np.zeros_like(A)
    for i in range(n): L[i][i] = 1.0

    for i in range(n):
        for j in range(n):
            if i <= j:
                s = sum(L[i][k] * U[k][j] for k in range(i))
                U[i][j] = A[i][j] - s
            elif i > j:
                s = sum(L[i][k] * U[k][j] for k in range(j))
                L[i][j] = (A[i][j] - s) / U[j][j]
    return L, U

n = 5
A = generate_symmetrical_matrix(10, 40, n)
A = increase_diag_elems(A, 3)
print(f'matrix A:\n{A}\n')

L, U = LU(A)
print(f'matrix L:\n{L}\n')
print(f'matrix U:\n{U}\n')
print(f'matrix L*U:\n{L @ U}\n')
lu_err = matrix_norm(A - L @ U)
print(f'LU decomposition error: {lu_err}\n')

matrix A:
[[199.32723742  13.36168261  14.50762551  27.14356867  11.42953568]
 [ 13.36168261 226.15556668  27.78905004  14.41624636  19.81820989]
 [ 14.50762551  27.78905004 218.94406972  19.03914996  11.64553106]
 [ 27.14356867  14.41624636  19.03914996 233.16450116  17.12253539]
 [ 11.42953568  19.81820989  11.64553106  17.12253539 180.04743606]]

matrix L:
[[1.         0.         0.         0.         0.        ]
 [0.0670339  1.         0.         0.         0.        ]
 [0.07278296 0.11904715 1.         0.         0.        ]
 [0.13617591 0.05592077 0.0724931  1.         0.        ]
 [0.05734056 0.08457806 0.03980314 0.06097998 1.        ]]

matrix U:
[[199.32723742  13.36168261  14.50762551  27.14356867  11.42953568]
 [  0.         225.25988095  26.81654728  12.59670702  19.05204351]
 [  0.           0.         214.69572828  15.56395871   8.54556415]
 [  0.           0.           0.         227.63550368  13.88120847]
 [  0.           0.           0.           0.         176.594059

РК 1 - Решение системы уравнения через LU-разложение

In [9]:
def solve_SLAU(L, U, b):
  n = len(b)

  y = np.zeros(n)
  for i in range(n):
    y[i] = b[i] - sum(L[i][k] * y[k] for k in range(0, i))

  y1 = np.linalg.solve(L, b)
  print(f'y: {y}, y1: {y1}')

  x = np.zeros(n)
  for i in range(n - 1, -1, -1):
    x[i] = (y[i] - sum(U[i][k] * x[k] for k in range(i + 1, n))) / U[i][i]

  x1 = np.linalg.solve(U, y)
  print(f'x: {x}, x1: {x1}')

  return x

correct_x = generate_vec(10, 40, n)
b = A @ correct_x
print(f'matrix A:\n{A}\n')
print(f'vector b: {b}')
L, U = LU(A)
x = solve_SLAU(L, U, b)
print(f'correct x: {correct_x}')
print(f'calculated x: {x}')
print(f'\nEuclid ||х* - x||: {get_diff(correct_x, x)}')
np.allclose(correct_x, x)

matrix A:
[[199.32723742  13.36168261  14.50762551  27.14356867  11.42953568]
 [ 13.36168261 226.15556668  27.78905004  14.41624636  19.81820989]
 [ 14.50762551  27.78905004 218.94406972  19.03914996  11.64553106]
 [ 27.14356867  14.41624636  19.03914996 233.16450116  17.12253539]
 [ 11.42953568  19.81820989  11.64553106  17.12253539 180.04743606]]

vector b: [5829.62365266 5298.73275634 3964.02439901 5585.40612878 5845.6549401 ]
y: [5829.62365266 4907.95033257 2955.44964909 4302.84571318 4716.25251252], y1: [5829.62365266 4907.95033257 2955.44964909 4302.84571318 4716.25251252]
x: [23.37645905 17.20002807 11.45051739 17.27377212 26.70674504], x1: [23.37645905 17.20002807 11.45051739 17.27377212 26.70674504]
correct x: [23.37645905 17.20002807 11.45051739 17.27377212 26.70674504]
calculated x: [23.37645905 17.20002807 11.45051739 17.27377212 26.70674504]

Euclid ||х* - x||: 7.32410687763558e-15


True

РК 1 - Вычисление определителя матрицы через LU-
разложение

In [3]:
def get_matrix_det(L, U):
  det = 1
  n = U.shape[0]
  for i in range(n):
    det *= U[i][i]
  return det

print(f'matrix det through liblaly: {np.linalg.det(A)}')
print(f'matrix det through LU decomposition: {get_matrix_det(L, U)}')

matrix det through liblaly: 1979895612248.6667
matrix det through LU decomposition: 1979895612248.669


РК 1 - Поиск обратной матрицы через LU-разложение

In [4]:
def get_inversed_matrix(L, U):
  n = L.shape[0]
  E = np.identity(n)
  INV = np.zeros((n, n), dtype=float)

  for i in range(n):
    y = np.zeros(n)
    x = np.zeros(n)
    for j in range(n):
      y[j] = E[j][i]
      for k in range(j):
        y[j] = y[j] - L[j][k] * y[k]
      y[j] = y[j] / L[j][j]

    for j in range(n - 1, -1, -1):
      x[j] = y[j]
      for k in range(j + 1, n):
        x[j] = x[j] - U[j][k] * x[k]
      x[j] = x[j] / U[j][j]

    INV[:, i] = x

  return INV

A_inv = get_inversed_matrix(L, U)
print(f'inversed matrix A through LU:\n{A_inv}\n')
print(f'inversed matrix A through lib:\n{np.linalg.inv(A)}\n')
err = matrix_norm(A_inv - np.linalg.inv(A))
print(f'error: {err}\n')

inversed matrix A through LU:
[[ 3.40240668e-03 -3.28605628e-04 -2.58360425e-04 -1.86417301e-04
  -1.58531940e-04]
 [-3.28605628e-04  3.26201552e-03 -7.46868687e-05 -2.76001373e-04
  -2.86426183e-04]
 [-2.58360425e-04 -7.46868687e-05  3.45830724e-03 -2.50982380e-04
  -3.59017778e-04]
 [-1.86417301e-04 -2.76001373e-04 -2.50982380e-04  4.44658397e-03
  -1.50362817e-04]
 [-1.58531940e-04 -2.86426183e-04 -3.59017778e-04 -1.50362817e-04
   3.13691035e-03]]

inversed matrix A through lib:
[[ 3.40240668e-03 -3.28605628e-04 -2.58360425e-04 -1.86417301e-04
  -1.58531940e-04]
 [-3.28605628e-04  3.26201552e-03 -7.46868687e-05 -2.76001373e-04
  -2.86426183e-04]
 [-2.58360425e-04 -7.46868687e-05  3.45830724e-03 -2.50982380e-04
  -3.59017778e-04]
 [-1.86417301e-04 -2.76001373e-04 -2.50982380e-04  4.44658397e-03
  -1.50362817e-04]
 [-1.58531940e-04 -2.86426183e-04 -3.59017778e-04 -1.50362817e-04
   3.13691035e-03]]

error: 1.0299920638612292e-18

